In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import KFold


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

../input/sample_submission.csv
../input/train.csv
../input/test.csv


In [2]:
def CRMSLE(truth, pred): 
    rmsle1 = (mean_squared_log_error(truth[:,0], pred[:,0]))**(1/2) 
    rmsle2 = (mean_squared_log_error(truth[:,1], pred[:,1]))**(1/2) 
    rmsle3 = (mean_squared_log_error(truth[:,2], pred[:,2]))**(1/2) 
    return (rmsle1 + rmsle2 + rmsle3)/3

In [3]:
sample_submission = pd.read_csv('../input/sample_submission.csv')
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [4]:
train.head()

,date_time,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,target_carbon_monoxide,target_benzene,target_nitrogen_oxides
0,2010-03-10 18:00:00,13.1,46.0,0.7578,1387.2,1087.8,1056.0,1742.8,1293.4,2.5,12.0,167.7
1,2010-03-10 19:00:00,13.2,45.3,0.7255,1279.1,888.2,1197.5,1449.9,1010.9,2.1,9.9,98.9
2,2010-03-10 20:00:00,12.6,56.2,0.7502,1331.9,929.6,1060.2,1586.1,1117.0,2.2,9.2,127.1
3,2010-03-10 21:00:00,11.0,62.4,0.7867,1321.0,929.0,1102.9,1536.5,1263.2,2.2,9.7,177.2
4,2010-03-10 22:00:00,11.9,59.0,0.7888,1272.0,852.7,1180.9,1415.5,1132.2,1.5,6.4,121.8


In [5]:
test.head()

,date_time,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5
0,2011-01-01 00:00:00,8.0,41.3,0.4375,1108.8,745.7,797.1,880.0,1273.1
1,2011-01-01 01:00:00,5.1,51.7,0.4564,1249.5,864.9,687.9,972.8,1714.0
2,2011-01-01 02:00:00,5.8,51.5,0.4689,1102.6,878.0,693.7,941.9,1300.8
3,2011-01-01 03:00:00,5.0,52.3,0.4693,1139.7,916.2,725.6,1011.0,1283.0
4,2011-01-01 04:00:00,4.5,57.5,0.4650,1022.4,838.5,871.5,967.0,1142.3


In [6]:
sample_submission.head()

,date_time,target_carbon_monoxide,target_benzene,target_nitrogen_oxides
0,2011-01-01 00:00:00,2.0,10.0,300.0
1,2011-01-01 01:00:00,2.0,10.0,300.0
2,2011-01-01 02:00:00,2.0,10.0,300.0
3,2011-01-01 03:00:00,2.0,10.0,300.0
4,2011-01-01 04:00:00,2.0,10.0,300.0


In [7]:
columns = test.columns[1:]
columns

Index(['deg_C', 'relative_humidity', 'absolute_humidity', 'sensor_1',
       'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5'],
      dtype='object')

In [8]:
X = train[columns].values
X_test = test[columns].values
target_0 = train['target_carbon_monoxide'].values.reshape(-1,1)
target_1 = train['target_benzene'].values.reshape(-1,1)
target_2 = train['target_nitrogen_oxides'].values.reshape(-1,1)

In [9]:
target_0.min()

0.1

In [10]:
target_1.min()


0.1

In [11]:
target_2.min()


1.9

In [12]:
target_0.max()

12.5

In [13]:
target_1.max()


63.7

In [14]:
target_2.max()


1472.3

In [15]:
train_oof = np.zeros((train.shape[0],3))
test_preds = np.zeros((test.shape[0],3))
train_oof.shape

(7111, 3)

In [16]:
test_preds.shape


(2247, 3)

In [17]:
%%time
n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(train)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target_0 = target_0[train_index]
    train_target_1 = target_1[train_index]
    train_target_2 = target_2[train_index]
    
    val_features = X[val_index]
    val_target_0 = target_0[val_index]
    val_target_1 = target_1[val_index]
    val_target_2 = target_2[val_index]
    
    model = Ridge(alpha=5000)
    model.fit(train_features, train_target_0)
    val_pred_0 = model.predict(val_features)
    train_oof[val_index,0] = val_pred_0.flatten()
    test_preds[:,0] += model.predict(X_test).flatten()/n_splits
    
    model = Ridge(alpha=5000)
    model.fit(train_features, train_target_1)
    val_pred_1 = model.predict(val_features)
    train_oof[val_index,1] = val_pred_1.flatten()
    test_preds[:,1] += model.predict(X_test).flatten()/n_splits
    
    model = Ridge(alpha=5000)
    model.fit(train_features, train_target_2)
    val_pred_2 = model.predict(val_features)
    train_oof[val_index,2] = val_pred_2.flatten()
    test_preds[:,2] += model.predict(X_test).flatten()/n_splits

Fitting fold 1
Fitting fold 2
Fitting fold 3
Fitting fold 4
Fitting fold 5
CPU times: user 90.4 ms, sys: 113 ms, total: 203 ms
Wall time: 43.2 ms


In [18]:
train_oof[:,0] = np.clip(train_oof[:,0], 0.12, None)
train_oof[:,1] = np.clip(train_oof[:,1], 0.12, None)
train_oof[:,2] = np.clip(train_oof[:,2], 2.50, None)

In [19]:
test_preds[:,0] = np.clip(test_preds[:,0], 0.12, None)
test_preds[:,1] = np.clip(test_preds[:,1], 0.12, None)
test_preds[:,2] = np.clip(test_preds[:,2], 2.50, None)

In [20]:
target = np.hstack([target_0, target_1, target_2])
target.shape

(7111, 3)

In [21]:
CRMSLE(target, train_oof)


0.41581139683024676

In [22]:
sample_submission[sample_submission.columns[1:]] = test_preds
sample_submission.head()

,date_time,target_carbon_monoxide,target_benzene,target_nitrogen_oxides
0,2011-01-01 00:00:00,2.000538,5.262982,351.196729
1,2011-01-01 01:00:00,2.925174,9.408482,499.158151
2,2011-01-01 02:00:00,2.307093,8.089543,424.119761
3,2011-01-01 03:00:00,2.513910,9.251186,421.426150
4,2011-01-01 04:00:00,2.038841,7.238631,375.058023


In [23]:
sample_submission.to_csv('../submissions/submission_rr_2.csv', index=False)
